# FloPy shapefile export demo
The goal of this notebook is to demonstrate ways to export model information to shapefiles.
This example will cover:
* basic exporting of information for a model, individual package, or dataset
* custom exporting of combined data from different packages
* general exporting and importing of geographic data from other sources

In [1]:
import os
import numpy as np
import pandas as pd
import flopy
from flopy.utils.reference import SpatialReference
fm = flopy.modflow

In [2]:
# set the output directory
outdir = 'tmp'
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# load an existing model
model_ws = "../data/freyberg"
m = fm.Modflow.load("freyberg.nam", model_ws=model_ws, verbose=False,
                               check=False, exe_name="mfnwt")

In [3]:
m.get_package_list()

['DIS', 'BAS6', 'LPF', 'WEL', 'RIV', 'RCH', 'OC', 'PCG']

### set the model `SpatialReference`
the spatial reference describes where the grid is located in a projected coordinate system (e.g. UTM)

In [4]:
m.sr = SpatialReference(delr=m.dis.delr.array, delc=m.dis.delc.array, 
                        xul=273170, yul=5088657, # model upper left corner in UTM coordinates
                        epsg=26916, # UTM zone 16 north
                        lenuni=1 # model length units (1 for feet, 2 for meters (default))
                       )

In [5]:
m.sr

xul:273170; yul:5088657; rotation:0; proj4_str:+proj=utm +zone=16 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ; units:meters; lenuni:1; length_multiplier:0.3048

## Declarative export using attached `.export()` methods
#### Export the whole model to a single shapefile

In [6]:
m.export('{}/model.shp'.format(outdir))

wrote tmp/model.shp


'tmp/model.shp'

### Export a package to a shapefile

In [7]:
m.wel.export('{}/wel.shp'.format(outdir))

wrote tmp/wel.shp


### Export a FloPy list or array object

In [8]:
m.lpf.hk

In [9]:
m.lpf.hk.export('{}/hk.shp'.format(outdir))

wrote tmp/hk.shp


In [10]:
m.riv.stress_period_data

In [11]:
m.riv.stress_period_data.export('{}/riv_spd.shp'.format(outdir))

wrote tmp/riv_spd.shp


### MfList.export() exports the whole grid by default, regardless of the locations of the boundary cells
`sparse=True` only exports the boundary cells in the MfList

In [12]:
m.riv.stress_period_data.export('{}/riv_spd.shp'.format(outdir), sparse=True)

wrote tmp/riv_spd.shp


In [13]:
m.wel.stress_period_data.export('{}/wel_spd.shp'.format(outdir), sparse=True)

wrote tmp/wel_spd.shp


## Ad-hoc exporting using `recarray2shp`
* The main idea is to create a recarray with all of the attribute information, and a list of geometry features (one feature per row in the recarray)
* each geometry feature is an instance of the `Point`, `LineString` or `Polygon` classes in `flopy.utils.geometry`. The shapefile format requires all the features to be of the same type.
* We will use pandas dataframes for these examples because they are easy to work with, and then convert them to recarrays prior to exporting.


In [14]:
from flopy.export.shapefile_utils import recarray2shp

### combining data from different packages
write a shapefile of RIV and WEL package cells

In [15]:
wellspd = pd.DataFrame(m.wel.stress_period_data[0])
rivspd = pd.DataFrame(m.riv.stress_period_data[0])
spd = wellspd.append(rivspd)
spd.head()

,cond,flux,i,iface,j,k,rbot,stage
0,NaN,-0.00820,8,0.0,15,0,NaN,NaN
1,NaN,-0.00410,10,0.0,12,0,NaN,NaN
2,NaN,-0.00390,19,0.0,13,0,NaN,NaN
3,NaN,-0.00083,25,0.0,9,0,NaN,NaN
4,NaN,-0.00072,28,0.0,5,0,NaN,NaN


##### create a list of Polygon features from the cell vertices stored in the SpatialReference object

In [16]:
from flopy.utils.geometry import Polygon

vertices = m.sr.get_vertices(spd.i, spd.j)
polygons = [Polygon(vrt) for vrt in vertices]

##### write the shapefile

In [17]:
recarray2shp(spd.to_records(), geoms=polygons,
             shpname='{}/bcs.shp'.format(outdir),
             epsg=m.sr.epsg)

wrote tmp/bcs.shp


### exporting other data
Suppose we have some well data with actual locations that we want to export to a shapefile

In [18]:
welldata = pd.DataFrame({'wellID': np.arange(0, 10),
                         'q': np.random.randn(10)*100 - 1000,
                         'x_utm': np.random.rand(10)*1000 + m.sr.xul,
                         'y_utm': m.sr.yul - np.random.rand(10)*3000})
welldata.head()

,q,wellID,x_utm,y_utm
0,-1004.276415,0,273476.608228,5.085864e+06
1,-907.415659,1,274091.346322,5.088361e+06
2,-923.543520,2,273215.410391,5.086950e+06
3,-889.842993,3,273676.977138,5.087234e+06
4,-1073.811703,4,273863.902903,5.087331e+06


##### convert the x, y coorindates to point features and then export

In [19]:
from flopy.utils.geometry import Point
geoms = [Point(x, y) for x, y in zip(welldata.x_utm, welldata.y_utm)]

recarray2shp(welldata.to_records(), geoms=geoms,
             shpname='{}/wel_data.shp'.format(outdir),
             epsg=m.sr.epsg)

wrote tmp/wel_data.shp


### Adding attribute data to an existing shapefile
Suppose we have a GIS coverage representing the river in the riv package

In [26]:
from flopy.utils.geometry import LineString 

### make up a linestring shapefile of the river reaches
i, j = m.riv.stress_period_data[0].i, m.riv.stress_period_data[0].j
x0 = m.sr.xcentergrid[i[0], j[0]]
x1 = m.sr.xcentergrid[i[-1], j[-1]]
y0 = m.sr.ygrid[i[0], j[0]]
y1 = m.sr.ygrid[i[-1]+1, j[-1]+1]
x = np.linspace(x0, x1, m.nrow+1)
y = np.linspace(y0, y1, m.nrow+1)
l0 = zip(list(zip(x[:-1], y[:-1])), list(zip(x[1:], y[1:])))
lines = [LineString(l) for l in l0]

rivdata = pd.DataFrame(m.riv.stress_period_data[0])
rivdata['reach'] = np.arange(len(lines))
lines_shapefile = '{}/riv_reaches.shp'.format(outdir)
recarray2shp(rivdata.to_records(index=False), geoms=lines,
             shpname=lines_shapefile,
             epsg=m.sr.epsg)

wrote tmp/riv_reaches.shp


#### read in the GIS coverage using `shp2recarray`
`shp2recarray` reads a shapefile into a numpy record array, which can easily be converted to a DataFrame

In [21]:
from flopy.export.shapefile_utils import shp2recarray

In [22]:
linesdata = shp2recarray(lines_shapefile)
linesdata = pd.DataFrame(linesdata)
linesdata.head()

,k,i,j,stage,cond,rbot,iface,reach,geometry
0,0,0,14,20.100000,0.05,20.00,0.0,0,<flopy.utils.geometry.LineString object at 0x1...
1,0,1,14,19.870001,0.05,19.75,0.0,1,<flopy.utils.geometry.LineString object at 0x1...
2,0,2,14,19.650000,0.05,19.50,0.0,2,<flopy.utils.geometry.LineString object at 0x1...
3,0,3,14,19.420000,0.05,19.25,0.0,3,<flopy.utils.geometry.LineString object at 0x1...
4,0,4,14,19.190001,0.05,19.00,0.0,4,<flopy.utils.geometry.LineString object at 0x1...


##### Suppose we have some flow information that we read in from the cell budget file

In [23]:
# make up some fluxes between the river and aquifer at each reach
q = np.random.randn(len(linesdata))+1
q

array([ 2.51373414,  1.66649271,  0.88489229,  1.25030968,  2.05674976,
        1.45243972,  0.99888104,  1.44134803,  0.76802466,  1.25265065,
       -0.00517002, -0.3408835 ,  0.53004571,  1.64510792, -0.91133883,
        1.00085903,  2.73890085,  0.77258356,  1.38275751,  0.55809936,
        1.6913228 ,  1.19534318,  0.90137371,  1.77253862,  1.44183576,
        2.48823491,  0.85183274,  0.26956319,  1.12190798, -0.48630765,
        2.07158711,  1.14827974,  0.29280159,  2.73028003,  1.3130897 ,
       -1.72419844,  1.46222895,  1.32117973,  1.49623908,  0.48105025])

##### Add reachs fluxes and cumulative flow to lines DataFrame

In [24]:
linesdata['qreach'] = q
linesdata['qstream'] = np.cumsum(q)

In [25]:
recarray2shp(linesdata.drop('geometry', axis=1).to_records(), 
             geoms=linesdata.geometry,
             shpname=lines_shapefile,
             epsg=m.sr.epsg)

wrote tmp/riv_reaches.shp
